In [3]:
import pandas as pd
from base64 import b64encode
import urllib.request, hashlib, requests, multiprocessing

In [4]:
url = "https://www.sec.gov/files/EDGAR_LogFileData_thru_Jun2017.html"
file = urllib.request.urlopen(url)
lst = list()
for line in file:
	decoded_line = line.decode("utf-8").strip()
	lst.append(decoded_line)
lst = lst[8:-2]
lst.sort()

Create a TSV file of URLS to transfer job:

In [5]:
def download_url(url, chunk_size=128, save_path='./tmp'):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)
    return save_path

def contentLength(link):
    site = urllib.request.urlopen('http://' + link)
    meta = site.info()
    return meta['Content-Length']

def checksum(link, hash_factory=hashlib.md5, chunk_num_blocks=128):
    url = 'http://' + link
    filename = download_url(url)
    h = hash_factory()
    with open(filename, 'rb') as f:
        for chunk in iter(lambda: f.read(chunk_num_blocks*h.block_size), b''): 
            h.update(chunk)
    return b64encode(h.digest())

def checksum_md5(link):
    url = 'http://' + link
    filename = download_url(url)
    md5 = hashlib.md5()
    with open(filename,'rb') as f: 
        for chunk in iter(lambda: f.read(8192), b''): 
            md5.update(chunk)
    return b64encode(md5.digest())

In [ ]:
df = pd.DataFrame()
for idx, lnk in enumerate(lst[:]):
    link, size, index = ( "TsvHttpData-1.0", None, None) if idx == 0 else ('http://' + lnk, contentLength(lnk), checksum(lnk))
    df = df.append([[link, size, index]], ignore_index=True)

df.to_csv('./file-name.tsv', sep='\t', index=False, header=False)